In [108]:
# Imports
import math
import numpy as np

In [145]:
# Speeds up 15x
DP_sim = {}

In [146]:
# Load in epinions dataset
trust_graph = {} # Source user -> List of trusted users
ratings = {} # Source user -> {movie : rating} map
rated_by = {} # Movie -> Set of users who have rated it

# Form trust graph
with open('../datasets/epinions/trust_data.txt') as in_file:
    for l in in_file.readlines():
        f, t, _ = [int(v) for v in l.split()]
        if f not in trust_graph:
            trust_graph[f] = [t]
        else:
            trust_graph[f].append(t)

# Map users to their ratings
with open('../datasets/epinions/ratings_data.txt') as in_file:
    for l in in_file.readlines():
        s, m, r = [int(v) for v in l.split()]
        # Fulfil ratings graph
        if s not in ratings:
            ratings[s] = {m : r}
        else:
            ratings[s][m] = r
        
        # Fulfil rated_by graph
        if m not in rated_by:
            rated_by[m] = set([s])
        else:
            rated_by[m].add(s)

In [147]:
# Returns the set of common users who have rated both movie i and j
def UC(i, j):
    return rated_by[i].intersection(rated_by[j])

In [148]:
# Returns the mean average rating given by a user
def r_(u):
    return sum(ratings[u]) / len(ratings[u])

In [149]:
# Returns the Pearson correlation of items i and j
def corr(i, j):
    uc = UC(i, j)
    
    def _numerator_term(u, i, j):
        mean = r_(u)
        return (ratings[u][i] - mean) * (ratings[u][j] - mean)
    
    def _denominator_term(u, i):
        return pow(ratings[u][i] - r_(u), 2)
    
    numerator = sum((_numerator_term(u, i, j) for u in uc))
    denominator_1 = math.sqrt(sum((_denominator_term(u, i) for u in uc)))
    denominator_2 = math.sqrt(sum((_denominator_term(u, i) for u in rated_by[i])))
    
    return numerator / (denominator_1 * denominator_2 + 1e-4) # Add a small delta to avoid division by zero

In [150]:
def sim(i, j):
    if (i,j) in DP_sim:
        return DP_sim[(i, j)]
    uc = UC(i, j)
    s = (1/(1 + pow(math.e, -len(uc)/2))) * corr(i,j)
    DP_sim[(i, j)] = s
    return s

In [151]:
def phi(current_user, item, k):
    sigmoid_scalar = 1 / (1 + pow(math.e, -k/2))
    m = -1
    if current_user not in ratings:
        return 1
    for j in ratings[current_user]:
        m = max(sim(item, j) * sigmoid_scalar, m)
    return m

In [152]:
import random
def random_walk(user, item):
    # Continue our random walk until broken
    current_user = user
    k = 0
    MAX_DEPTH = 6
    while True:
        # If we go deeper than 6, cancel our walk
        if k >= MAX_DEPTH:
            return None
        
        # If the current user already has rated the item, return it
        if current_user in ratings and item in ratings[current_user]:
            return ratings[current_user][item]
        
        # With probability phi we cancel our random walk
        p = phi(current_user, item, k)
        if random.random() < p:
            if current_user not in ratings:
                return None
            return random.choice(list(ratings[current_user].values()))
        
        # Otherwise continue our walk
        if current_user in trust_graph:
            current_user = random.choice(trust_graph[current_user])
            k += 1
        else:
            return None

In [153]:
%%timeit
random_walk(23083, 38760)

126 µs ± 38.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [155]:
def produce_recommendation(user, item):
    random_walks = []
    err = 0
    sigma_sq_i = 1
    e = 0.05
    
    def _calc_sig():
        mean = sum(random_walks) / len(random_walks)
        return sum([pow(r_i - mean, 2) for r_i in random_walks]) / len(random_walks)
    
    while True:
        rw = random_walk(user, item)
        if rw is None:
            err += 1
            if err >= 10000:
                return -1
            continue
        random_walks.append(rw)
        sigma_sq_i1 = _calc_sig()
        
        if abs(sigma_sq_i1 - sigma_sq_i) <= e:
            return round(np.mean(random_walks))

In [156]:
produce_recommendation(23083, 38760)

4